In [1]:
import requests

def get_velib_data():
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=20'

    try:
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return {'error': f'Failed to fetch data. Status code: {response.status_code}'}

    except Exception as e:
        return {'error': f'An error occurred: {str(e)}'}

if __name__ == '__main__':
    velib_data = get_velib_data()
    
    if 'error' in velib_data:
        print(f"Error: {velib_data['error']}")
    else:
        print("Velib Data:")
        print(velib_data)


Velib Data:
{'total_count': 1471, 'results': [{'stationcode': '16107', 'name': 'Benjamin Godard - Victor Hugo', 'is_installed': 'OUI', 'capacity': 35, 'numdocksavailable': 25, 'numbikesavailable': 9, 'mechanical': 4, 'ebike': 5, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T13:17:04+00:00', 'coordonnees_geo': {'lon': 2.275725, 'lat': 48.865983}, 'nom_arrondissement_communes': 'Paris', 'code_insee_commune': None}, {'stationcode': '9020', 'name': 'Toudouze - Clauzel', 'is_installed': 'OUI', 'capacity': 21, 'numdocksavailable': 20, 'numbikesavailable': 1, 'mechanical': 0, 'ebike': 1, 'is_renting': 'OUI', 'is_returning': 'OUI', 'duedate': '2023-11-19T13:13:04+00:00', 'coordonnees_geo': {'lon': 2.3373600840568547, 'lat': 48.87929591733507}, 'nom_arrondissement_communes': 'Paris', 'code_insee_commune': None}, {'stationcode': '14111', 'name': 'Cassini - Denfert-Rochereau', 'is_installed': 'OUI', 'capacity': 25, 'numdocksavailable': 18, 'numbikesavailable': 4, 'mechanical'

In [2]:
from flask import Flask

app = Flask(__name__)
app.config["DEBUG"] = True

@app.route('/', methods=['GET'])
def home():
    return "<h1>Distant Reading Archive</h1><p>This site is a prototype API for distant reading of science fiction novels.</p>"

if __name__ == '__main__':
    app.run(use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2023 15:13:51] "GET / HTTP/1.1" 200 -


In [14]:
from flask import Flask, render_template, jsonify, request
import requests
import math
import sqlite3  # Ajoutez cette ligne pour importer la bibliothèque sqlite3

app = Flask(__name__, template_folder='C:\\Users\\kraha\\OneDrive\\Bureau\\appFLASK')

DB_NAME = 'velib_data.db'

def create_table():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS velib_stations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            stationcode TEXT,
            name TEXT,
            capacity INTEGER,
            numdocksavailable INTEGER,
            numbikesavailable INTEGER,
            mechanical INTEGER,
            ebike INTEGER,
            is_renting TEXT,
            is_returning TEXT,
            duedate TEXT,
            lon REAL,
            lat REAL
        )
    ''')

    conn.commit()
    conn.close()

create_table()

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

import os

# Obtenez le chemin du répertoire actuel du notebook
notebook_dir = os.getcwd()

@app.route('/')
def map():
    # Construisez le chemin absolu vers le fichier HTML en utilisant le répertoire du notebook
    html_path = os.path.join(notebook_dir, 'index.html')
    return render_template(html_path)



@app.route('/api/velib_data', methods=['POST'])
def velib_data():
    user_location = request.json

    # Récupérer les données en temps réel à partir de l'API Velib
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=20'
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()['records']
    else:
        return jsonify({'error': 'Unable to fetch Velib data'})

    # Filtrer les données pour ne renvoyer que celles à proximité de l'utilisateur
    nearby_stations = []
    for record in data:
        station = record['fields']
        distance = haversine(user_location['lat'], user_location['lon'], station['coordonnees_geo']['lat'], station['coordonnees_geo']['lon'])
        if distance <= 5.0:  # Exemple : inclure les stations dans un rayon de 5 kilomètres
            nearby_stations.append({
                'stationcode': station['stationcode'],
                'name': station['name'],
                'capacity': station['capacity'],
                'numdocksavailable': station['numdocksavailable'],
                'numbikesavailable': station['numbikesavailable'],
                'mechanical': station['mechanical'],
                'ebike': station['ebike'],
                'is_renting': station['is_renting'],
                'is_returning': station['is_returning'],
                'duedate': station['duedate'],
                'coordonnees_geo': station['coordonnees_geo']
            })

    return jsonify({'nearby_stations': nearby_stations})

if __name__ == '__main__':
    app.run(use_reloader=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-11-19 20:07:29,070] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\kraha\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\kraha\AppData

In [15]:
from flask import Flask, render_template, jsonify, request
import requests
import math
import sqlite3

app = Flask(__name__, template_folder='C:\\Users\\kraha\\OneDrive\\Bureau\\appFLASK\\templates')

DB_NAME = 'velib_data.db'

def create_table():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS velib_stations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            stationcode TEXT,
            name TEXT,
            capacity INTEGER,
            numdocksavailable INTEGER,
            numbikesavailable INTEGER,
            mechanical INTEGER,
            ebike INTEGER,
            is_renting TEXT,
            is_returning TEXT,
            duedate TEXT,
            lon REAL,
            lat REAL
        )
    ''')

    conn.commit()
    conn.close()

create_table()

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

import os

# Obtenez le chemin du répertoire actuel du notebook
notebook_dir = os.getcwd()

@app.route('/')
def map():
    # Utilisez simplement le nom du fichier modèle sans le chemin complet
    return render_template('index.html')

@app.route('/api/velib_data', methods=['POST'])
def velib_data():
    user_location = request.json

    # Récupérer les données en temps réel à partir de l'API Velib
    api_url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=20'
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()['records']
    else:
        return jsonify({'error': 'Unable to fetch Velib data'})

    # Filtrer les données pour ne renvoyer que celles à proximité de l'utilisateur
    nearby_stations = []
    for record in data:
        station = record['fields']
        distance = haversine(user_location['lat'], user_location['lon'], station['coordonnees_geo']['lat'], station['coordonnees_geo']['lon'])
        if distance <= 5.0:  # Exemple : inclure les stations dans un rayon de 5 kilomètres
            nearby_stations.append({
                'stationcode': station['stationcode'],
                'name': station['name'],
                'capacity': station['capacity'],
                'numdocksavailable': station['numdocksavailable'],
                'numbikesavailable': station['numbikesavailable'],
                'mechanical': station['mechanical'],
                'ebike': station['ebike'],
                'is_renting': station['is_renting'],
                'is_returning': station['is_returning'],
                'duedate': station['duedate'],
                'coordonnees_geo': station['coordonnees_geo']
            })

    return jsonify({'nearby_stations': nearby_stations})

# Add a route for serving the HTML page
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run(use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
